In [ ]:
# TODO Tuto for using / implementing a variant in ACETONE

# ACETONE tutorial #2

**Implementing and using other versions of a layer**

Efficiency is a key aspect in the embedded sector, with each code being specifically adpated to a terget. As such, we need to be able to chose the implementation of each layer.

In this notebook, we'll explain how to use specific version of a layer in ACETONE.

* When running this notebook on Colab, we need to install ACETONE 
* If you run this notebook locally, run it in the environment in which you installed ACETONE

In [ ]:
# Cleaning the working environment
from pathlib import Path
from os import remove, listdir
files_directories = []

for directory in files_directories:
    if directory.exists():
        for file in listdir(directory):
            remove(directory / file)

## Imports

In [ ]:
import numpy as np

import acetone_nnet

## Generating the first code

